# Libraires

In [14]:
# import json file called config
import json
with open('config.json') as f:
    config = json.load(f)
TOKENIZER_ID = config['TOKENIZER_ID']
nrows = config['nrows']

In [15]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import PassiveAggressiveClassifier #used to train the model
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, confusion_matrix, roc_auc_score, roc_curve # this is used to evaluate the model.
from sklearn.model_selection import train_test_split # to separate the dataset.
from sklearn.feature_extraction.text import TfidfVectorizer #used for the transformation of text data.
import pickle as pkl #file to save and load a model.
from sklearn.model_selection import GridSearchCV #pour l'optimisation des hyperparametres.
import time #pour étudier le temps d'execution d'entrainement, de notre meilleure modèle.
import psutil #pour calculer la mémoire.
import os #pour calculer le temps d'execution.
import joblib


# Load Data

In [35]:
data=pd.read_csv('title_text.csv',nrows=250)
#check if isFake column has unique values
print(data['isFake'].value_counts())

isFake
True     133
False    117
Name: count, dtype: int64


In [36]:
data

,title,text,subject,date,isFake
0,New Zealand Green Party leader says wants to f...,WELLINGTON (Reuters) - New Zealand Green Party...,worldnews,"September 23, 2017",False
1,Lunatic MSNBC Reporter: “A paper trail leads d...,This guy is a joke! Scott Dworkin: A paper tr...,politics,"Jul 22, 2017",True
2,Fox News Host Leaks That Spicer Is Getting Fi...,Already demonstrating her inability to keep he...,News,"May 16, 2017",True
3,OOPS! Lindsey Vonn Gets Hit With Big Dose Of K...,"Only 3 days ago, Lindsey Vonn told CNN that sh...",politics,"Dec 9, 2017",True
4,The Weather Channel Has Responded To Trump Wi...,"Today, Donald Trump yanked us out of the Paris...",News,"June 1, 2017",True
...,...,...,...,...,...
245,Canada's Trudeau tells Trump a NAFTA pullout w...,OTTAWA (Reuters) - Canadian Prime Minister Jus...,politicsNews,"April 27, 2017",False
246,Veteran Spy: Trump Trading Secret Info With R...,The connection between Donald Trump s presiden...,News,"October 31, 2016",True
247,Factbox: Key court cases shed light on U.S. Su...,(Reuters) - Neil Gorsuch has taken part in mor...,politicsNews,"March 21, 2017",False
248,BREAKING: Iran Publicly Humiliates Obama…Unvei...,A clear violation of Obama s lopsided deal w...,Government News,"Jan 5, 2016",True


In [37]:
#we make everything lower.
data['text']=data['text'].apply(lambda x: x.lower())
data

,title,text,subject,date,isFake
0,New Zealand Green Party leader says wants to f...,wellington (reuters) - new zealand green party...,worldnews,"September 23, 2017",False
1,Lunatic MSNBC Reporter: “A paper trail leads d...,this guy is a joke! scott dworkin: a paper tr...,politics,"Jul 22, 2017",True
2,Fox News Host Leaks That Spicer Is Getting Fi...,already demonstrating her inability to keep he...,News,"May 16, 2017",True
3,OOPS! Lindsey Vonn Gets Hit With Big Dose Of K...,"only 3 days ago, lindsey vonn told cnn that sh...",politics,"Dec 9, 2017",True
4,The Weather Channel Has Responded To Trump Wi...,"today, donald trump yanked us out of the paris...",News,"June 1, 2017",True
...,...,...,...,...,...
245,Canada's Trudeau tells Trump a NAFTA pullout w...,ottawa (reuters) - canadian prime minister jus...,politicsNews,"April 27, 2017",False
246,Veteran Spy: Trump Trading Secret Info With R...,the connection between donald trump s presiden...,News,"October 31, 2016",True
247,Factbox: Key court cases shed light on U.S. Su...,(reuters) - neil gorsuch has taken part in mor...,politicsNews,"March 21, 2017",False
248,BREAKING: Iran Publicly Humiliates Obama…Unvei...,a clear violation of obama s lopsided deal w...,Government News,"Jan 5, 2016",True


# Modification du text en données numériques.
Nous partons du principe que les données ont étés clean et que nous pouvons nous concerntrer à créer un model.

Comme les informations principales pour définir des Fake news et des vrai news, se base sur le texte et le titre, nous devons nous focaliser sur ces informations. Il faut ainsi que nous transformons le text et le titre en données afin que notre model puisse utiliser des données numériques pour trouver la bonne réponse. Nous allons utiliser la methode de bert-.
# Bert tokenizer

In [38]:
from transformers import BertTokenizer, BertModel


X = data['text']

# Create the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_bert = BertModel.from_pretrained('bert-base-uncased')

# Set the maximum sequence length
max_seq_length = 512

# Truncate or pad the tokenized sequences
X = X.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_seq_length, truncation=True, padding='max_length'))
print(X)

0      [101, 8409, 1006, 26665, 1007, 1011, 2047, 341...
1      [101, 2023, 3124, 2003, 1037, 8257, 999, 3660,...
2      [101, 2525, 14313, 2014, 13720, 2000, 2562, 20...
3      [101, 2069, 1017, 2420, 3283, 1010, 17518, 385...
4      [101, 2651, 1010, 6221, 8398, 10963, 2149, 204...
                             ...                        
245    [101, 8166, 1006, 26665, 1007, 1011, 3010, 353...
246    [101, 1996, 4434, 2090, 6221, 8398, 1055, 4883...
247    [101, 1006, 26665, 1007, 1011, 6606, 2175, 286...
248    [101, 1037, 3154, 11371, 1997, 8112, 1055, 884...
249    [101, 3956, 1055, 5193, 2704, 2003, 6183, 3805...
Name: text, Length: 250, dtype: object


In [39]:
y = data['isFake'].astype(int) #make it numerical
print(y.value_counts())

isFake
1    133
0    117
Name: count, dtype: int64


In [40]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length") # Tokenize the text
    outputs = model_bert(**inputs) # Utiliser l'embedding du premier token ([CLS]) comme représentation du texte
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

In [41]:
# we separate the trdata into training test and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [42]:
X_train = np.array(X_train.tolist())
X_train.shape

(160, 512)

In [43]:
X_test = np.array(X_test.tolist())

X_test.shape,y_test.shape

((50, 512), (50,))

# Model Passive Agressive

Quand on parle d'un model passive aggressive, nous parlons d'un algorithme linéaire de classification et de regression. C'est un algorithme d'apprentissage en ligne, Il traite les données séquentiellement, ajustant le prédicteur à chaque nouvelle instance pour améliorer la prédiction future.

Le modèle se base sur deux principes:
- **Passive**: Si notre modèle prédit une instance juste, alors les poids et les biais ne seront pas changés. L'état actuelle de notre modèle sera maintenu.
- **Aggressive**: Si notre modèle fait une mauvaise prédiction sur une instance ressus, les poids et biais de notre modèle seront changés par le paramètre de régularisation **C**, le but étant de minimiser l'erreur. Il change les paramètres pour que la prédiction actuelle soit 'just'.

Le modèle utilise par défaut la fonction de cout **hinge**. Voici comment mathématiquement les poids et bias seront changés:

$$ w_{\text{new}} = w + \alpha y x $$
$$ b_{\text{new}} = b + \alpha y $$

où **w** est le vecteur de poids, **b** est le biais, **x** est le vecteur de caractéristiques de l'instance mal classée, **y** est l'étiquette correcte de l'instance, et **α** est le taux d'apprentissage calculé en fonction de l'erreur et du paramètre de régularisation **C**

Comme nous traitons des données de texte, et que nous devons faire une classification binaire, il ce va de dire que les text vont avoir des pattern differents, en utilisant le model passive aggressive, nous pouvons entrainer le model à définir une méthode pour les idenfifier.

In [44]:
# we build our model: C is the regulation parameter, and denotes the penalization the model will make on an incorrect prediciton.
model = PassiveAggressiveClassifier(C = 0.2, 
                                    random_state = 42,
                                    early_stopping=True,# we enable the early stopping procedure.
                                    n_iter_no_change=3, #number of iterations before early stopping
                                    verbose=True,
                                    loss='hinge', #fonction de cout
                                   )


In [45]:
filenm = 'PassiveAggressiveClassifier_model.pickle'

In [46]:
#on entraine le model sur les données d'entrainement.
model.fit(X_train,y_train)
#Create or open a file with write-binary mode and save the model to it

-- Epoch 1
Norm: 0.01, NNZs: 512, Bias: 0.000052, T: 144, Avg. loss: 11.817556
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.01, NNZs: 512, Bias: 0.000054, T: 288, Avg. loss: 4.624221
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.01, NNZs: 512, Bias: 0.000055, T: 432, Avg. loss: 1.997314
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.01, NNZs: 512, Bias: 0.000055, T: 576, Avg. loss: 1.127731
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.01, NNZs: 512, Bias: 0.000055, T: 720, Avg. loss: 0.503602
Total training time: 0.00 seconds.
Convergence after 5 epochs took 0.00 seconds


PassiveAggressiveClassifier(C=0.2, early_stopping=True, n_iter_no_change=3,
                            random_state=42, verbose=True)

In [49]:
#on sort les predictions
#Step 2: Open the saved file with read-binary mode
y_pred=model.predict(X_test)

# Optimisation

Pour optimiser le model nous allons modifier les paramètres suivants:
- **C (Le paramètre de régularisation)**: Un des paramètres les plus importants de l'algorithme passive aggressive. Il est utilisé pour controler la marge d'erreur que notre modèle est près à accepté. Si on lui donne une valeur de plus en plus élevé il se peut que le modèle suraprenne, car il s'adapte mieux aux données d'entrainement. Ainsi une valeur plus faible serait favorable. Nous allons tester C avec les valeurs suivantes: **[0.01,0.05,0.1,0.5,1]**

In [50]:
params_grid={'C':[0.01,0.05,0.1,0.5,1]}

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import make_scorer, accuracy_score


# Initialize the Passive Aggressive Classifier
model = PassiveAggressiveClassifier(random_state=42,early_stopping=True)

# On creer un scorer pour le grid search 
scorer = make_scorer(accuracy_score)

# Initialisation de la GridSearch pour trouver le meilleur C.
grid_search = GridSearchCV(estimator=model, param_grid=params_grid, scoring=scorer, cv=5)

# On lance les multiples entrainements.
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=PassiveAggressiveClassifier(early_stopping=True,
                                                   random_state=42),
             param_grid={'C': [0.01, 0.05, 0.1, 0.5, 1]},
             scoring=make_scorer(accuracy_score, response_method='predict'))

In [52]:
# Get the best combination of parameters
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")


Best parameters: {'C': 0.01}
Best score: 0.58125


# Model Evaluation

Afin d'évaluer le modèle nous faisons une représentation graphique des résultats obtenus. Nous allons étudier les metrics suivants:
- le **temps d'entrainement** que le modèle a besoin, ainsi que son **taux de mémoire (Mo)**.
- **matrice de confusion**, qui est un excellent choix d'évaluation de performance, car nous sommes dans le cas d'une classification binaire. 
- **L'accuracy** du model.
- **La précision**
- **recall**
- **F1-score**.
- **ROC-AUC score**

src: https://www.v7labs.com/blog/performance-metrics-in-machine-learning#h2

In [53]:
#creation du modèle avec les meilleurs paramètres
best_model=PassiveAggressiveClassifier(C=best_params['C'],
                                       early_stopping=True,
                                       verbose=True,
                                       random_state=42)

Nous voullons étudier **le temps d'entrainement (en h/m/s)** et **l'utilisation de la mémoire (Mo)** que le modèle a besoin.

In [54]:
def second_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return int(hours), int(minutes), int(seconds)

def bytes_to_Mo(mem_bytes):
    mem_kb = mem_bytes / 1024  # Convertir en kilooctets
    mem_mb = mem_kb / 1024  # Convertir en mégaoctets
    return mem_mb

In [56]:
#lancemenet de l'enrestristrement de la mémoire.
process = psutil.Process(os.getpid())
mem_before_bytes = process.memory_info().rss

#lancement de l'enregistrement du temps d'entrainement.
start_time = time.time() 
# entrainement du modèle.
best_model.fit(X_train,y_train)
end_time = time.time()

#le temps écoulé 
elapsed_time= end_time - start_time

print(f"Temps d'entrainement (h | m | s) : {second_to_hms(elapsed_time)[0]} | {second_to_hms(elapsed_time)[1]} | {second_to_hms(elapsed_time)[2]}")

mem_after_bytes = process.memory_info().rss
#on convertit les bytes en Mo.
mem_bytes=mem_after_bytes - mem_before_bytes


print(f"Utilisation de la mémoire (Mo) : {bytes_to_Mo(mem_bytes)}")

-- Epoch 1
Norm: 0.01, NNZs: 512, Bias: 0.000052, T: 144, Avg. loss: 11.817556
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.01, NNZs: 512, Bias: 0.000054, T: 288, Avg. loss: 4.624221
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.01, NNZs: 512, Bias: 0.000055, T: 432, Avg. loss: 1.997314
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.01, NNZs: 512, Bias: 0.000055, T: 576, Avg. loss: 1.127731
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.01, NNZs: 512, Bias: 0.000055, T: 720, Avg. loss: 0.503602
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 0.01, NNZs: 512, Bias: 0.000056, T: 864, Avg. loss: 0.213721
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 0.01, NNZs: 512, Bias: 0.000056, T: 1008, Avg. loss: 0.137438
Total training time: 0.01 seconds.
Convergence after 7 epochs took 0.01 seconds
Temps d'entrainement (h | m | s) : 0 | 0 | 0
Utilisation de la mémoire (Mo) : 0.00390625


In [57]:
#Create or open a file with write-binary mode and save the model to it
pickle = pkl.dump(best_model, open(filenm, 'wb'))

In [59]:
#optention des prédictions de ce modèle.
y_pred=best_model.predict(X_train)

Nous déterminons la **matrice de confusion**.

In [60]:
#creation de la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

#représentation graphique du résultat du meilleure model
sns.heatmap(cm, annot=True)
plt.xlabel('Les labels prédits')
plt.ylabel('Les vrais labels')
plt.title('Matrice de confusion')
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [50, 160]

In [ ]:
print(f"le modèle a donc {cm[0][0]} instances vrai Positives et {cm[1][1]} instances de Vrai négatives, {cm[1][0]} instances de Faux Positives et {cm[0][1]} instances de Faux négatives. ")

Nous allons avant tout calculer **la précision globale (accuracy)** de notre modèle. 

In [ ]:
acc=accuracy_score(y_pred,y_test)
print(f"Notre modèle a une précision globale de {round(acc*100,2)}%, ce qui signifie qu'il prédit correctement les classes des instances dans {round(acc*100,2)}% des cas.")

on calcul le **precision**, c'est la capacité du modèle à correctement identifier les instances que nous avons prédits. Donc de prédire qu'une News vrai soit vraie.

In [ ]:
precision=precision_score(y_test,y_pred)
print(f"notre model a une precision de {round(precision*100,2)}% . lorsqu'il prédit une classe comme positive, il a raison dans {round(precision*100,2)}% des cas.")

on calcul le **recall**, ce qui es le pourcentage de Vrai positif **(dans notre cas les Vrai news)** que notre modèle arrive à prédire.

In [ ]:
recall=recall_score(y_test,y_pred)
print(f"notre model arrive a détecter {round(recall*100,2)}%. Donc sur l'ensemble des vrai positives, le modèle parvient à en identifier correctement {round(recall*100,2)}% des cas.")

On calcule le **F1-score**, c'est une combinaison de la précision et du recall, en calculant la moyenne harmonique entre les deux. Son but est **d'étudier l'équilibre entre la precision et le recall.**

In [ ]:
f1score=f1_score(y_test,y_pred)
print(f"notre model a un F1-score de {round(f1score*100,2)}%, ce qui signifie que il y a un excellent equilibre entre le recall et la précision.")

## Courbe ROC-AUC
Dans le contexte d'une classification binaire, il est essentiel d'utiliser la courbe ROC-AUC pour évaluer notre modèle. Nous allons examiner deux aspects :

- **ROC (Receiver Operating Characteristic)**: Cette courbe illustre le rapport entre le taux de vrais positifs et le taux de faux positifs. Elle permet de visualiser la capacité du modèle à discriminer entre les deux classes.

- **AUC (Area Under the Curve)**: C'est une mesure qui résume la courbe ROC en calculant l'aire sous celle-ci et au-dessus de la diagonale qui va du coin inférieur gauche au coin supérieur droit du graphique. Un score AUC proche de 1 indique que notre modèle fait des prédictions presque parfaites. En revanche, un score proche de 0,5 suggère que nos prédictions sont principalement due au hasard, ce qui signifie que le modèle n'apprend pas de manière efficace.

L'objectif de ces mesures est de déterminer à quel point notre modèle est efficace dans la tâche de classification binaire.

On définit le score ROC-AUC.

In [ ]:
roc_auc=roc_auc_score(y_test,y_pred)
roc_auc

In [ ]:
print(f"Le modèle a donc une score ROC-AUC de {roc_auc} ce qui est très proche de 1, donc le modèle a une excellente capacité à différencier entres les Fake et les vrais news.")

Avec une valeur aussi élevée, nous pouvons être confiants dans le fait que notre modèle est extrêmement précis et fiable pour notre tâche de classification, avec très peu de faux positifs et de faux négatifs. Cela indique que notre modèle fait des erreurs de prédiction minimales. Nous allons maintenant procéder à la représentation graphique de la courbe ROC-AUC.

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Taux de Faux Positives (%)')
plt.ylabel('Taux de Vrai Positives (%)')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

L'axe des abscisses (axe x) représente le taux de faux positifs, c'est-à-dire le pourcentage de vraies nouvelles que notre modèle a incorrectement classifiées comme fausses (Fake News).

L'axe des ordonnées (axe y) indique le taux de vrais positifs, qui correspond au pourcentage de fausses nouvelles que notre modèle a correctement identifiées comme telles.

La courbe montante vers le coin supérieur gauche du graphique suggère que le modèle parvient à maintenir un taux élevé de vrais positifs tout en minimisant le taux de faux positifs, ce qui est le comportement idéal pour un classificateur. 
Cette représentation représente la robustesse du mdoèle.

# Test de performances sur des données générées par Téléchargé ailleurs

In [61]:
validation_data=pd.read_csv("data/validation_data.csv")
validation_data.head()

,Unnamed: 0,text,isFake
0,0,"Daniel Greenfield, a Shillman Journalism Fello...",False
1,1,Google Pinterest Digg Linkedin Reddit Stumbleu...,False
2,2,U.S. Secretary of State John F. Kerry said Mon...,True
3,3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",False
4,4,It's primary day in New York and front-runners...,True


In [62]:
validation_data=validation_data[['text','isFake']]
validation_data

,text,isFake
0,"Daniel Greenfield, a Shillman Journalism Fello...",False
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,False
2,U.S. Secretary of State John F. Kerry said Mon...,True
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",False
4,It's primary day in New York and front-runners...,True
...,...,...
6330,The State Department told the Republican Natio...,True
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,False
6332,Anti-Trump Protesters Are Tools of the Oligar...,False
6333,"ADDIS ABABA, Ethiopia —President Obama convene...",True


In [64]:
# Prepare the data
validation_data_X = validation_data['text'].apply(lambda x: x.lower())  # the text data
validation_data_y = validation_data['isFake']

# Tokenize the data using the BERT tokenizer
encoded_inputs = tokenizer(validation_data_X.tolist(), padding=True, truncation=True, max_length=max_seq_length, return_tensors='np')

# Get the input IDs and attention masks
input_ids = encoded_inputs['input_ids']
attention_masks = encoded_inputs['attention_mask']

# Use the trained model to make predictions on the dataset
predictions = best_model.predict(input_ids)


In [65]:
confusion_matrix(validation_data_y,predictions)

array([[1511, 1653],
       [1377, 1794]], dtype=int64)

In [66]:
accuracy_score(validation_data_y,predictions)

0.5217048145224941

# Conclusion

les évaluations et analyses approfondies de notre modèle confirment qu'il est extrêmement efficace et fiable pour l'identification des fake news. Ses performances élevées sur plusieurs métriques clés (accuracy,precision,recall,F1-score et ROC-AUC) démontrent sa capacité à fournir des prédictions précises et équilibrées, confortant notre choix d'utiliser ce modèle pour cette tache.